In [1]:
import numpy as np
import pandas as pd
import pickle
%matplotlib inline
from IPython.display import display

In [2]:
# bids = pd.read_csv('bids.csv')
bids = pickle.load(open('bids.pkl'))

In [3]:
print bids.shape
display(bids.head())

(7656329, 9)


,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


In [4]:
bidders = bids.groupby('bidder_id')

### 针对国家、商品单一特征多类别转换为多个独立特征进行统计

In [5]:
cates = (bids['merchandise'].unique()).tolist()
countries = (bids['country'].unique()).tolist()

def dummy_coun_cate(group):
    coun_cate = dict.fromkeys(cates, 0)
    coun_cate.update(dict.fromkeys(countries, 0))
    for cat, value in group['merchandise'].value_counts().iteritems():
        coun_cate[cat] = value

    for c in group['country'].unique():
        coun_cate[c] = 1

    coun_cate = pd.Series(coun_cate)
    return coun_cate


In [6]:
bidder_coun_cate = bidders.apply(dummy_coun_cate)

In [7]:
display(bidder_coun_cate.describe())
bidder_coun_cate.to_csv('coun_cate.csv')

,ad,ae,af,ag,al,am,an,ao,ar,at,...,vc,ve,vi,vn,ws,ye,za,zm,zw,zz
count,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,...,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000,6609.000000
mean,0.002724,0.205629,0.054774,0.001059,0.048570,0.023907,0.000303,0.036314,0.120442,0.052655,...,0.000605,0.033591,0.000303,0.130882,0.001967,0.040551,0.274474,0.067181,0.069753,0.000757
std,0.052121,0.404191,0.227555,0.032530,0.214984,0.152770,0.017395,0.187085,0.325502,0.223362,...,0.024596,0.180186,0.017395,0.337297,0.044311,0.197262,0.446283,0.250354,0.254750,0.027497
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


同样的，对于每个用户需要统计他对于自己每次竞拍行为的时间间隔情况

In [8]:
def bidder_interval(group):
    time_diff = np.ediff1d(group['time'])
    bidder_interval = {}
    if len(time_diff) == 0:
        diff_mean = 0
        diff_std = 0
        diff_median = 0
        diff_zeros = 0
    else:
        diff_mean = np.mean(time_diff)
        diff_std = np.std(time_diff)
        diff_median = np.median(time_diff)
        diff_zeros = time_diff.shape[0] - np.count_nonzero(time_diff)
    bidder_interval['tmean'] = diff_mean
    bidder_interval['tstd'] = diff_std
    bidder_interval['tmedian'] = diff_median
    bidder_interval['tzeros'] = diff_zeros
    bidder_interval = pd.Series(bidder_interval)
    return bidder_interval

In [9]:
bidder_inv = bidders.apply(bidder_interval)

In [10]:
display(bidder_inv.describe())
bidder_inv.to_csv('bidder_inv.csv')

,tmean,tmedian,tstd,tzeros
count,6.609000e+03,6.609000e+03,6.609000e+03,6609.000000
mean,2.933038e+12,1.860285e+12,3.440901e+12,122.986231
std,8.552343e+12,7.993497e+12,6.512992e+12,3190.805229
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,1.192853e+10,2.578947e+09,1.749995e+09,0.000000
50%,2.641139e+11,5.726316e+10,5.510107e+11,0.000000
75%,1.847456e+12,6.339474e+11,2.911282e+12,0.000000
max,7.610295e+13,7.610295e+13,3.800092e+13,231570.000000


### 按照用户-拍卖场分组进一步分析
之前的统计是按照用户进行分组，针对各个用户从整体上针对竞标行为统计其各项特征，下面根据拍卖场来对用户进一步细分，看一看每个用户在不同拍卖场的行为模式,类似上述按照用户分组来统计各个用户的各项特征，针对用户-拍卖场结对分组进行统计以下特征

* 基本计数统计，针对各个用户在各个拍卖场统计设备、国家、ip、url、商品类别、竞标次数等特征的数目作为新的特征
* 时间间隔统计：统计各个用户在各个拍卖场每次竞拍的时间间隔的 均值、方差、中位数和0值
* 针对商品类别、国家进一步转化为多类别进行统计

In [11]:
def auc_features_count(group):
    time_diff = np.ediff1d(group['time'])
    
    if len(time_diff) == 0:
        diff_mean = 0
        diff_std = 0
        diff_median = 0
        diff_zeros = 0
    else:
        diff_mean = np.mean(time_diff)
        diff_std = np.std(time_diff)
        diff_median = np.median(time_diff)
        diff_zeros = time_diff.shape[0] - np.count_nonzero(time_diff)

    row = dict.fromkeys(cates, 0)
    row.update(dict.fromkeys(countries, 0))

    row['devices_c'] = group['device'].unique().shape[0]
    row['countries_c'] = group['country'].unique().shape[0]
    row['ip_c'] = group['ip'].unique().shape[0]
    row['url_c'] = group['url'].unique().shape[0]
#     row['merch_c'] = group['merchandise'].unique().shape[0]
    row['bids_c'] = group.shape[0]
    row['tmean'] = diff_mean
    row['tstd'] = diff_std
    row['tmedian'] = diff_median
    row['tzeros'] = diff_zeros

    for cat, value in group['merchandise'].value_counts().iteritems():
        row[cat] = value

    for c in group['country'].unique():
        row[c] = 1

    row = pd.Series(row)
    return row

In [12]:
bidder_auc = bids.groupby(['bidder_id', 'auction']).apply(auc_features_count)

In [13]:
bidder_auc.to_csv('bids_auc.csv')

In [14]:
print bidder_auc.shape

(382336, 218)
